# Bushfire Detection Model Development
Using the Edge Impulse Python SDK with TensorFlow and Keras to train model to detect bushfires vs. no bushfire within a given image.

#Install Dependencies

In [3]:
!python -m pip install tensorflow==2.12.0 edgeimpulse

# Import Packages

In [14]:
from tensorflow import keras
import edgeimpulse as ei
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os

# Setup

In [15]:
# Settings
ei.API_KEY = "ei_c90f6349e4f2624274eff30ee840a3e6412c3135d1651ee15515404d290ce842"
labels = ["nofire", "fire"]
num_classes = len(labels)
deploy_filename = "bushfire_detection_model_cpp.zip"

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Define paths for your dataset
train_val_directory = 'drive/MyDrive/forest_fire/Training and Validation'
test_directory = 'drive/MyDrive/forest_fire/Testing'

# Create an ImageDataGenerator for training and validation
train_val_datagen = ImageDataGenerator(
    rescale=1.0 / 255,        # Normalize pixel values
    validation_split=0.2      # Use 20% of the training data for validation
)

# Create generators for training and validation data
train_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),   # Resize images to 128x128
    batch_size=32,             # Number of images to return in each batch
    class_mode='binary',       # Binary classification (fire or no fire)
    subset='training',          # Set as training data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

validation_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',       # Binary classification
    subset='validation',        # Set as validation data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

Found 1467 images belonging to 2 classes.
Found 365 images belonging to 2 classes.


In [19]:
# Create an ImageDataGenerator for testing (no split needed)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create a generator for the test data
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',       # Binary classification
    shuffle=False,             # Don't shuffle test data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

Found 68 images belonging to 2 classes.


In [20]:
# Build your model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Adjust epochs as needed
)

Epoch 1/10
45/45 [==============================] - 247s 5s/step - loss: 0.7381 - accuracy: 0.8606 - val_loss: 0.3428 - val_accuracy: 0.8778
Epoch 2/10
45/45 [==============================] - 35s 786ms/step - loss: 0.1275 - accuracy: 0.9519 - val_loss: 0.2532 - val_accuracy: 0.9091
Epoch 3/10
45/45 [==============================] - 36s 790ms/step - loss: 0.0892 - accuracy: 0.9707 - val_loss: 0.2340 - val_accuracy: 0.9148
Epoch 4/10
45/45 [==============================] - 34s 750ms/step - loss: 0.0580 - accuracy: 0.9847 - val_loss: 0.2409 - val_accuracy: 0.9318
Epoch 5/10
45/45 [==============================] - 36s 792ms/step - loss: 0.0390 - accuracy: 0.9909 - val_loss: 0.2605 - val_accuracy: 0.9176
Epoch 6/10
45/45 [==============================] - 41s 920ms/step - loss: 0.0241 - accuracy: 0.9930 - val_loss: 0.2729 - val_accuracy: 0.9233
Epoch 7/10
45/45 [==============================] - 33s 734ms/step - loss: 0.0114 - accuracy: 0.9993 - val_loss: 0.2643 - val_accuracy: 0.9318
E

In [22]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_accuracy:.2f}')

2/2 [==============================] - 8s 7s/step - loss: 0.3205 - accuracy: 0.9531
Test Accuracy: 0.95


In [ ]:
# List the available profile target devices
# ei.model.list_profile_devices()

In [23]:
# Estimate the RAM, ROM, and inference time for our model on the target hardware family
try:
    profile = ei.model.profile(model=model,
                               device='openmv-h7p')
    print(profile.summary())
except Exception as e:
    print(f"Could not profile: {e}")

KeyboardInterrupt: 

In [25]:
# Define the output and input types for your model
labels = ["nofire", "fire"]  # Update the labels to match your classification task

model_output_type = ei.model.output_type.Classification(labels=labels)
model_input_type = ei.model.input_type.ImageInput()  # Assuming images are the input type

# Define the filename for the deployment package
deploy_filename = "bushfire_detection_model.zip"

# Try to deploy the model directly as a C++ library in a .zip file
deploy_bytes = None
try:
    deploy_bytes = ei.model.deploy(
        model=model,  # The trained model
        model_output_type=model_output_type,
        model_input_type=model_input_type,
        deploy_target='zip'  # Export as a .zip file with C++ library
    )
    print("Model deployment successful.")
except Exception as e:
    print(f"Could not deploy: {e}")

# Write the deployed model bytes to a file
if deploy_bytes:
    with open(deploy_filename, 'wb') as f:
        f.write(deploy_bytes.getvalue())
    print(f"Model exported as {deploy_filename}")
else:
    print("Deployment failed; no deployable bytes were generated.")

Could not deploy: 
----------------
You specified a Classification model with 2 labels but the model has 1 labels.
----------------

For more information see https://docs.edgeimpulse.com/reference/python-sdk or ask a question at https://forum.edgeimpulse.com/
Deployment failed; no deployable bytes were generated.


# Model Specifications
Code to check if model has been trained correctly

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 127008)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                8128576   
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 8,129,537
Trainable params: 8,129,537
Non-trainable params: 0
____________________________________________